# Solarlog Data Analyse mit Clustering
This Kernal shows how to connect to Impala in order to load data directly from HAL PROD or STAG.

## Import

More information about impyla see here: https://github.com/cloudera/impyla.

In [1]:
from impala.dbapi import connect 
from impala.util import as_pandas
import pandas as pd # For Data Handling
import numpy as np  # data processing, CSV file I/O (e.g. pd.read_csv)
import math as mt
import datetime as dt # To convert timestamp data
import matplotlib   #for plots
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(color_codes=True)
matplotlib.style.use('ggplot')
%matplotlib inline

## Day -7

In [2]:
today = dt.date.today()
week_ago = today - dt.timedelta(days=7)
epoch_last7 = week_ago.strftime('%s')

## Connect to DataHost wh-02 (172.16.10.102)
Querry benötigt ca. 45s + 39.29s für das refresh = 1.5 min

In [3]:
conn = connect(host='172.16.10.102', port=21050)
cur = conn.cursor()
cur.execute('refresh solarlog_roh')
cur.execute('SELECT * FROM solarlog_roh where `timestamp` > CAST('+epoch_last7+' AS INT) order by `timestamp` limit 100000')
dfp = as_pandas(cur)
cur.close()
conn.close()

In [4]:
len(dfp)

14104

In [5]:
dfp.head()

,balance_flag,battery,battery_as_meter,city,consumption_meter,countrycode,cur_code,cur_consumption_percent,cur_consumption_percent_out,cur_consumption_watt,...,timestamp,version,weather_available,weather_copyright,weather_error_msg,yield_lifetime,yield_month,yield_today,yield_year,zipcode
0,earnings,False,False,,True,,113,0.0,0 %,0,...,1520636402,1.3,1,,,1220624,166841,0,802556,
1,earnings,False,False,,True,,116,0.0,0 %,0,...,1520636402,1.3,1,,,633891,213245,0,633891,
2,earnings,True,False,,True,,113,0.0,0 %,0,...,1520636402,1.3,1,,,6369010,187964,0,896610,
3,earnings,False,False,,True,,113,0.0,0 %,0,...,1520636402,1.3,1,,,7737067,270502,0,1321360,
4,earnings,False,False,,True,,116,0.0,0 %,0,...,1520636402,1.3,1,,,251860202,1605808,0,6544912,


In [6]:
# Change epochtime from timestamp to date time
dfp1 = pd.DataFrame() # New Data Frame
# Convert to datetime and add 1h corret time shift by collecting data
dfp1['datetime'] = dfp.timestamp.apply(dt.datetime.utcfromtimestamp) + pd.Timedelta(hours=1)
dfp1.sort_values(['datetime'])
dfp1['value_watt'] = dfp.cur_yield_watt
dfp1['name'] = dfp.plantname

In [7]:
pd.Timedelta(hours=1)

Timedelta('0 days 01:00:00')

In [8]:
dfp1.name.value_counts()

Meyer Rolf, Baar                      1763
Himmelrichstrasse 1ab, Baar           1763
Maechler Paul, Allenwinden            1763
Haller Roman, Rifferswil              1763
Loosmann Johannes, Hausen am Albis    1763
Winsun AG, Steg                       1763
Pfadiheim Baar                        1763
Betschart René, Steinhausen           1763
Name: name, dtype: int64

In [9]:
dfp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14104 entries, 0 to 14103
Data columns (total 3 columns):
datetime      14104 non-null datetime64[ns]
value_watt    14104 non-null int64
name          14104 non-null object
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 275.5+ KB


In [10]:
dfp1.groupby('name').describe().value_watt

,count,mean,std,min,25%,50%,75%,max
name,,,,,,,,
"Betschart René, Steinhausen",1763.0,976.531480,1749.124790,0.0,0.0,0.0,1238.0,9556.0
"Haller Roman, Rifferswil",1763.0,1291.974475,2113.054978,0.0,0.0,0.0,1940.5,11233.0
"Himmelrichstrasse 1ab, Baar",1763.0,1348.865570,2390.157423,0.0,0.0,0.0,1863.5,11063.0
"Loosmann Johannes, Hausen am Albis",1763.0,494.592172,1003.188958,0.0,0.0,0.0,540.5,5001.0
"Maechler Paul, Allenwinden",1763.0,977.470788,1962.248984,0.0,0.0,0.0,1051.0,9590.0
"Meyer Rolf, Baar",1763.0,995.209302,1861.364061,0.0,0.0,0.0,1233.0,8777.0
Pfadiheim Baar,1763.0,851.081679,1679.589965,0.0,0.0,0.0,1001.5,9746.0
"Winsun AG, Steg",1763.0,7116.849121,11475.385292,0.0,0.0,0.0,11134.0,50394.0


In [11]:
df1 = dfp1[dfp1['name'] == 'Pfadiheim Baar'].copy()
df2 = dfp1[dfp1['name'] == 'Meyer Rolf, Baar'].copy()
df3 = dfp1[dfp1['name'] == 'Winsun AG, Steg'].copy()
df4 = dfp1[dfp1['name'] == 'Himmelrichstrasse 1ab, Baar'].copy()
df5 = dfp1[dfp1['name'] == 'Betschart René, Steinhausen'].copy()
df6 = dfp1[dfp1['name'] == 'Loosmann Johannes, Hausen am Albis'].copy()
df7 = dfp1[dfp1['name'] == 'Haller Roman, Rifferswil'].copy()
#df8 = dfp1[dfp1['name'] == 'Maechler Paul, Allenwinden']

In [12]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True) #need to use plotly offline

data = Data([Scatter(x=df1.datetime, y=df1.value_watt, name='Pfadiheim Baar'),
             Scatter(x=df2.datetime, y=df2.value_watt, name='Meyer Rolf, Baar'),
             Scatter(x=df3.datetime, y=df3.value_watt, name='Winsun AG, Steg'),
             Scatter(x=df4.datetime, y=df4.value_watt, name='Himmelrichstrasse 1ab, Baar'),
             Scatter(x=df5.datetime, y=df5.value_watt, name='Betschart René, Steinhausen'),
             Scatter(x=df6.datetime, y=df7.value_watt, name='Loosmann Johannes, Hausen am Albis'),
             Scatter(x=df7.datetime, y=df7.value_watt, name='Haller Roman, Rifferswil')])


layout = Layout(
    title='Anlageleistung über 7 Tage',
    yaxis=dict(title='Leistung in [kW]'),
    xaxis=dict(title='Datum')
)

fig = Figure(data=data, layout=layout)

iplot(fig, filename = 'basic-line')

## Lade Solar Inventar Tabelle
Die Daten der Winsun Anlage in der inverntar_solar Tabelle werden direkt über die Solarlog API in HDFS konsumiert.

In [13]:
conn = connect(host='172.16.10.102', port=21050)
cur = conn.cursor()
cur.execute('refresh inventory_solar')
cur.execute('SELECT * FROM inventory_solar')
dfis = as_pandas(cur)
cur.close()
conn.close()
#To see all tables tip only dfp without .head()
dfp1['datetime'] = dfp.timestamp.apply(dt.datetime.utcfromtimestamp) + pd.Timedelta(hours=1)
dfp1.sort_values(['datetime'])
dfp1['value_watt'] = dfp.cur_yield_watt
dfp1['name'] = dfp.plantnamedfis

TTransportException: TTransportException(type=1, message="Could not connect to ('172.16.10.102', 21050)")

In [ ]:
#Using timestamp in epoch time
dfc = pd.DataFrame()
dfc['datetime'] = dfp.timestamp
dfc.sort_values(['datetime'])
dfc['value_watt'] = dfp.cur_yield_watt
dfc['name'] = dfp.plantname



dfc1 = dfc[dfc['name'] == 'Pfadiheim Baar'].copy()
dfc2 = dfc[dfc['name'] == 'Meyer Rolf, Baar'].copy()
dfc3 = dfc[dfc['name'] == 'Winsun AG, Steg'].copy()
dfc4 = dfc[dfc['name'] == 'Himmelrichstrasse 1ab, Baar'].copy()
dfc5 = dfc[dfc['name'] == 'Betschart René, Steinhausen'].copy()
dfc6 = dfc[dfc['name'] == 'Loosmann Johannes, Hausen am Albis'].copy()
dfc7 = dfc[dfc['name'] == 'Haller Roman, Rifferswil'].copy()
dfc1.drop('name', axis=1, inplace=True)
dfc2.drop('name', axis=1, inplace=True)
dfc3.drop('name', axis=1, inplace=True)
dfc4.drop('name', axis=1, inplace=True)
dfc5.drop('name', axis=1, inplace=True)
dfc6.drop('name', axis=1, inplace=True)
dfc7.drop('name', axis=1, inplace=True)
dfc1.head()

In [ ]:
timeSeries = pd.DataFrame()
df = pd.DataFrame
plt.figure(figsize=(25, 10))
for df in [dfc2,dfc4,dfc5]:
    timeSeries = timeSeries.append(df)
    ax = sns.tsplot( data=df.value_watt, err_style="unit_traces")

plt.show()

In [ ]:
plt.matshow(timeSeries.corr())

In [ ]:
new = timeSeries[timeSeries.index > 1600]
new.info

In [ ]:
from scipy import stats
import scipy.cluster.hierarchy as hac

import sys
sys.setrecursionlimit(100000)

# Do the clustering
Z = hac.linkage(timeSeries, method='single', metric='correlation')

# Plot dendogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
hac.dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()